# Caclulating rolling returns over different horizons

In [ ]:
library(tidyverse)
library(ggthemes)
library(odbc)
library(plutoR)
library(quantmod)
library(PerformanceAnalytics)

options("scipen"=999)
options(stringsAsFactors = FALSE)
options(repr.plot.width=16, repr.plot.height=8)

source("config.R")
source("goofy/plot.common.R")
source("goofy/misc.common.R")

indices <- Indices()

In [ ]:
numMonths <- 12 * 10 #10 years, a typical "long-term" buy and hold horizon
indexName <- "NIFTY 50 TR" #always use total return index where feasible

trIndex <- data.frame(indices$NseTimeSeries() %>%
    filter(NAME == indexName) %>%
    select(TIME_STAMP, CLOSE))

trXts <- xts(trIndex$CLOSE, as.Date(trIndex$TIME_STAMP))
monthlyTrReturns <- monthlyReturn(trXts)

#ignore the first and last months
monthlyTrReturns <- monthlyTrReturns[-1]
monthlyTrReturns <- monthlyTrReturns[-nrow(monthlyTrReturns)]

In [ ]:
#caculate rolling returns
rollingRets <- rollapply(monthlyTrReturns, numMonths, Return.annualized)
rollingRets <- na.omit(rollingRets)*100

rollingRetDf <- data.frame(rollingRets)
names(rollingRetDf) <- c('R')
rollingRetDf$T <- index(rollingRets)

In [ ]:
#plot rolling returns
firstDate <- first(index(rollingRets))
lastDate <- last(index(rollingRets))
xAxisTicks <- seq(from=firstDate, to=lastDate, length.out=10)

ggplot(rollingRetDf, aes(x=T, y=R)) +
    theme_economist() +
    geom_line() +
    scale_x_date(breaks = xAxisTicks) +
    labs(x='', y='%', color='', title=sprintf("%s - %d month rolling returns", indexName, numMonths), 
         subtitle=sprintf("[%s:%s]", firstDate, lastDate)) +
    annotate("text", x=lastDate, y=min(rollingRets), label = "@StockViz", hjust=1.1, vjust=-1.1, col="white", cex=6, fontface = "bold", alpha = 0.8)

This notebook was created using [pluto](http://pluto.studio). Learn more [here](https://github.com/shyams80/pluto)